Set Up Google Drive and Import Libraries

In [ ]:
# Mount Google Drive to access files
from google.colab import drive
drive.mount('/content/drive')

# Set the base directory to your chatbot folder in Google Drive
BASE_DIR = '/content/drive/MyDrive/CHAT-BOT'

# Verify the files are accessible
import os
os.listdir(BASE_DIR)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['Untitled0.ipynb',
 'setup.sh',
 '.setup.sh.un~',
 'requirements.txt',
 'README.md',
 '.chatbot.py.un~',
 'chatbot.py',
 '.train.py.un~',
 'train.py',
 'main.py',
 '.main.py.un~',
 '.preprocess.py.un~',
 'preprocess.py',
 'utils.py',
 '.utils.py.un~',
 'model.py',
 '.model.py.un~',
 'train_model.py',
 '.train_model.py.un~',
 '.train_model1.py.un~',
 'train_model1.py',
 'data',
 '__pycache__',
 '.git',
 'models',
 'chatbot_model.h5']

Import Dependencies and Define Functions

In [ ]:
import json
import random
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


Load and Preprocess the Data

In [ ]:
# Load intents data
with open(f"{BASE_DIR}/data/intents.json") as file:
    intents = json.load(file)

# Prepare the training data
def preprocess_data(intents):
    patterns = []
    tags = []
    for intent in intents['intents']:
        for pattern in intent['patterns']:
            patterns.append(pattern)
            tags.append(intent['tag'])

    # Convert tags to numerical values
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(tags)

    # Vectorize patterns
    vectorizer = TfidfVectorizer(max_features=2000)
    X = vectorizer.fit_transform(patterns).toarray()

    return X, y, vectorizer, label_encoder

X, y, vectorizer, label_encoder = preprocess_data(intents)


Define and Train the Model

In [ ]:
# Build model
model = Sequential([
    Dense(128, input_shape=(X.shape[1],), activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(len(set(y)), activation='softmax')
])

# Compile model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train model
history = model.fit(X, y, epochs=100, batch_size=8, verbose=1)
model.save(f"{BASE_DIR}/chatbot_model.h5")


Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0189 - loss: 4.5909
Epoch 2/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0289 - loss: 4.5556
Epoch 3/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0379 - loss: 4.4993
Epoch 4/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0532 - loss: 4.4228
Epoch 5/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0541 - loss: 4.3399
Epoch 6/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0413 - loss: 4.2354
Epoch 7/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0692 - loss: 4.0894
Epoch 8/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0669 - loss: 3.9771
Epoch 9/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0749 - loss: 3.9501
Epoch 10/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0991 - loss: 3.8594
Epoch 11/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1506 - loss: 3.6706
Epoch 12/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1218 - lo

Implement Chatbot Prediction

In [ ]:
# Function to predict class
def predict_class(message):
    vectorized_message = vectorizer.transform([message]).toarray()
    prediction = model.predict(vectorized_message)
    tag_index = np.argmax(prediction)
    tag = label_encoder.inverse_transform([tag_index])[0]
    return tag

# Get response based on intent tag
def get_response(tag, intents_json):
    for intent in intents_json["intents"]:
        if intent["tag"] == tag:
            return random.choice(intent["responses"])

# Main chatbot function
def chatbot_response(message):
    tag = predict_class(message)
    return get_response(tag, intents)


Run the Chatbot

In [ ]:
print("Start talking with the bot! Type 'quit' to end.")

while True:
    user_input = input("You: ")
    if user_input.lower() == "quit":
        break
    response = chatbot_response(user_input)
    print(f"Chatbot: {response}")



Start talking with the bot! Type 'quit' to end.
You: who are you 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Chatbot: I'm Odai, your Personal Therapeutic AI Assistant. How are you feeling today
You: what are you do 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Chatbot: I'm Odai, your Personal Therapeutic AI Assistant. How are you feeling today
You: well am not doing so well 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Chatbot: Bye! Come back again.
You: can you help me ?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Chatbot: I'm sorry to hear that. I'm doing my best to help
You: i am feeling weak
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Chatbot: Why do you think you feel this way?
You: my stomach
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Chatbot: I am really sorry to hear that. I am here to help you with grief, anxiety and anything else you may feel at this time.
You: i feel sad
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Chatbot: How long have you been feeling this way?
You: about two hours
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms

Overwriting /content/drive/MyDrive/CHAT-BOT/main.py
